**thuật toán vertical apriori**

In [ ]:
import numpy as np
def compute_vertical_bitvector_data(data):
    #---build item to idx mapping---#
    idx = 0
    item2idx = {}
    for transaction in data:
        for item in transaction:
            if not item in item2idx:
                item2idx[item] = idx
                idx += 1
    idx2item = { idx : item for item, idx in item2idx.items() }
    #---build vertical data---#
    vb_data = np.zeros((len(item2idx), len(data)), dtype=int)
    for trans_id, transaction in enumerate(data):
        for item in transaction:
            vb_data[item2idx[item], trans_id] = 1

    return vb_data, idx2item

def compute_L1(data, idx2item, num_trans, min_support):
	L1 = []
	support_list = {}
	for idx, bit_list in enumerate(data):
		support = np.sum(bit_list) / num_trans
		if support >= min_support:
			support_list[frozenset([idx2item[idx]])] = bit_list
			L1.append([idx2item[idx]])
	return list(map(frozenset, sorted(L1))), support_list

def compute_LK(LK_, support_list, k, num_trans, min_support):
    LK = []
    supportK = {}
    for i in range(len(LK_)):
        for j in range(i+1, len(LK_)):  # enumerate all combinations in the Lk-1 itemsets
            L1 = sorted(list(LK_[i])[:k-2])
            L2 = sorted(list(LK_[j])[:k-2])
            if L1 == L2: # if the first k-1 terms are the same in two itemsets, calculate the intersection support

                union = np.multiply(support_list[LK_[i]], support_list[LK_[j]])
                union_support = np.sum(union) / num_trans
                if union_support >= min_support:
                    new_itemset = frozenset(sorted(list(LK_[i] | LK_[j])))
                    if new_itemset not in LK:
                        LK.append(new_itemset)
                        supportK[new_itemset] = union
    return sorted(LK), supportK

def eclat(data, min_support, iterative=False,block=None, thread=None):

	num_trans = float(len(data))

	#---iterative method---#


	vb_data, idx2item = compute_vertical_bitvector_data(data)
	L1, support_list = compute_L1(vb_data, idx2item, num_trans, min_support)
	L = [L1]
	k = 1

	while True:
			print('Running Eclat: the %i-th iteration with %i itemsets in L%i...' % (k, len(L[-1]), k))
			k += 1
			LK, supportK = compute_LK(L[-1], support_list, k, num_trans, min_support)

			if len(LK) == 0:
				L = [sorted([tuple(sorted(itemset)) for itemset in LK]) for LK in L]
				support_list = dict((tuple(sorted(k)), np.sum(v)) for k, v in support_list.items())
				print('Running Eclat: the %i-th iteration. Terminating ...' % (k-1))
				break
			else:
				L.append(LK)
				support_list.update(supportK)
	return L, support_list



In [ ]:
data = [['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
    ['Wine', 'Bread', 'Butter', 'Milk'],
    ['Bread', 'Butter', 'Milk'],
    ['Chips', 'Butter', 'Apple'],
    ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
    ['Wine', 'Chips', 'Milk'],
    ['Wine', 'Chips', 'Bread', 'Butter', 'Apple'],
    ['Wine', 'Chips', 'Milk'],
    ['Wine', 'Bread', 'Apple'],
    ['Wine', 'Bread', 'Butter', 'Milk'],
    ['Chips', 'Bread', 'Butter', 'Apple'],
    ['Wine', 'Butter', 'Milk', 'Apple'],
    ['Wine', 'Chips', 'Bread', 'Butter', 'Milk'],
    ['Wine', 'Bread', 'Milk', 'Apple'],
    ['Wine', 'Bread', 'Butter', 'Milk', 'Apple'],
    ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
    ['Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
    ['Chips', 'Butter', 'Milk', 'Apple'],
    ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
    ['Wine', 'Bread', 'Butter', 'Milk', 'Apple'],
    ['Wine', 'Chips', 'Bread', 'Milk', 'Apple'],
    ['Chips', 'Bread', 'Milk']
]

L, support_list = eclat(data, min_support=3)

print('Frequent itemsets:')
for k in range(len(L)):
    for itemset in L[k]:
        print('{}: {}'.format(list(itemset), support_list[itemset]))


Running Eclat: the 1-th iteration with 0 itemsets in L1...
Running Eclat: the 1-th iteration. Terminating ...
Frequent itemsets:


**KIỂM TRA BẰNG THƯ VIỆN pyECLAT**

In [ ]:
!pip install pyECLAT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df=pd.read_excel('/content/drive/MyDrive/DataMining/csdl_tuan5.xlsx',header=None)
df.head().T

,0,1,2,3,4
0,Wine,Wine,NaN,NaN,Wine
1,Chips,NaN,NaN,Chips,Chips
2,Bread,Bread,Bread,NaN,Bread
3,Butter,Butter,Butter,Butter,Butter
4,Milk,Milk,Milk,NaN,Milk
5,Apple,NaN,NaN,Apple,Apple


In [ ]:
from pyECLAT import ECLAT
import pandas as pd

my_eclat = ECLAT(df, verbose=True)
rule_indices, rule_supports = my_eclat.fit(min_support=3, max_combination=2, min_combination=2)



100%|██████████| 6/6 [00:00<00:00, 1066.62it/s]


Combination 2 by 2


0it [00:00, ?it/s]


In [ ]:
rule_supports

{}